In [1]:
import pymdroutines as pmdr
import pandas as pd
import evaluate as e
import random as rand

In [2]:
evids =pd.read_json("data\\release_evidences.json")
evids1 = pd.read_json("data\\release_evidences.json")
conds  = pd.read_json('data\\release_conditions.json')
evids1 = evids1.T
print(evids1.columns)
# evids['numeric_part'] = evids['name'].apply(lambda x: int(x.split('_')[1]))
# sorted_evids = evids.sort_values(by='numeric_part')
# print(sorted_evids['name'].to_list())

Index(['name', 'code_question', 'question_fr', 'question_en', 'is_antecedent',
       'default_value', 'value_meaning', 'possible-values', 'data_type'],
      dtype='object')


In [3]:
conds = conds.T
conds.drop(['cond-name-fr', 'antecedents','cond-name-eng','icd10-id'], axis=1, inplace=True)
print(conds.columns)


Index(['condition_name', 'symptoms', 'severity'], dtype='object')


In [4]:
cond_to_evi = {}
int_to_cond ={}
for i in range(len(conds)):
   condition=  conds.iloc[i]
   name =condition['condition_name']
   cond_to_evi[name] = set(condition['symptoms'].keys())
   int_to_cond[i]=name



In [5]:
import pickle as p
with open('trained_model.bin', 'rb') as f:
    rf =  p.load(f)
    cols  = p.load(f)
print(len(cols))

914


In [6]:
multi_evi = pmdr.gen_multichoice_features(evids)[1]
ordinal_evi_sc= pmdr.gen_singlechoice_features(evids)[1]
categorical_evi_sc = pmdr.gen_singlechoice_features(evids)[2]
binary_evi =pmdr.gen_binary_features(evids)
# print(len(ordinal_evi_sc+  categoical_evi_sc+   multi_evi+      binary_evi),sep ='\n'*3)
all_evis = multi_evi+ordinal_evi_sc+categorical_evi_sc+binary_evi
print(rand.choice(all_evis))
# print(len(ordinal_evi_sc),len(categoical_evi_sc),len(multi_evi),len(binary_evi),sep ='\n'*3)



E_173


In [7]:
import thompsonsampling as ts


In [8]:
picked_bandit = ts.pick_bandit(49) # Pick disease with the current highest possibility (using beta sampling).
disease = int_to_cond[picked_bandit] 
symp_set = cond_to_evi[disease]
enquiry_chosen = rand.choice(list(symp_set)) # Choose a random symptom of that disease to enquire about disease.
# print(disease,enquiry_chosen,symp_set)
question_data = evids1.loc[enquiry_chosen,["question_en",'possible-values','value_meaning']]
print(question_data)
feat_group_classify = lambda enquiry_chosen: 1 if enquiry_chosen in categorical_evi_sc else (0 if enquiry_chosen in binary_evi else ( 3 if enquiry_chosen in multi_evi else 2 )) 
question_group_val = feat_group_classify(enquiry_chosen)


# if 

question_en               Does the pain radiate to another location?
possible-values    [V_123, V_14, V_15, V_16, V_17, V_18, V_19, V_...
value_meaning      {'V_123': {'fr': 'nulle part', 'en': 'nowhere'...
Name: E_57, dtype: object


In [9]:
import importlib
importlib.reload(pmdr)
print(question_group_val)
pmdr.disp(question_group_val, question_data["question_en"],question_data["value_meaning"],question_data["possible-values"])

3
V_123 nowhere
V_14 iliac wing(R)
V_15 iliac wing(L)
V_16 groin(R)
V_17 groin(L)
V_19 axilla(L)
V_18 axilla(R)
V_20 tonsil(R)
V_21 tonsil(L)
V_22 anus
V_26 back of the neck
V_25 back of head
V_30 biceps(R)
V_31 biceps(L)
V_32 mouth
V_33 thyroid cartilage
V_34 ankle(R)
V_35 ankle(L)
V_36 clitoris
V_37 coccyx
V_38 cervical spine
V_39 thoracic spine
V_40 lumbar spine
V_41 commissure(R)
V_42 commissure(L)
V_49 iliac crest(R)
V_50 iliac crest(L)
V_51 thigh(R)
V_52 thigh(L)
V_57 lower teeth(R)
V_58 lower teeth(L)
V_59 upper teeth(R)
V_60 upper teeth(L)
V_67 finger (index)(R)
V_68 finger (index)(L)
V_69 finger (middle)(R)
V_70 finger (middle)(L)
V_63 finger (ring finger)(R)
V_64 finger (ring finger)(L)
V_65 finger (little finger)(R)
V_66 finger (little finger)(L)
V_62 top of the head
V_194 shoulder(R)
V_195 shoulder(L)
V_197 epigastric
V_27 forearm(R)
V_28 forearm(L)
V_76 dorsal aspect of the hand(R)
V_77 dorsal aspect of the hand(L)
V_72 dorsal aspect of the foot(R)
V_73 dorsal aspect of th